# Google News Scraping



In [ ]:
import os
import time
import urllib.request
from multiprocessing import Pool

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
NUM = 30

key = pd.read_csv("/content/keyword.txt", encoding="gbk", names=["keyword"])
keywords = []
[keywords.append(key["keyword"][x]) for x in range(len(key))]
keywords

['meta']

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Creating directory. " + directory)

In [ ]:
def google_search(keyword):
    createFolder("./" + keyword + "_search_results")

    # chromedriver = 'chromedriver.exe'
    # driver = webdriver.Chrome(chromedriver)
    driver = webdriver.Chrome("chromedriver")
    driver.implicitly_wait(3)

    print("search", keyword)
    links = []
    times = []
    titles = []
    intros = []
    page = 1
    while True:
        driver.get(
            f"https://www.google.com/search?tbm=nws&num={NUM}&start={(page - 1) * 20}&tbs=cdr%3A1%2Ccd_min%3A11%2F1%2F2021%2Ccd_max%3A2%2F15%2F2022&q="
            + keyword
        )
        # driver.get(f'https://www.google.com/search?tbm=nws&num={NUM}&start={(page - 1) * 20}&as_qdr=w9&q=' + keyword)
        # driver.get(f'https://www.google.com/search?tbm=nws&num={NUM}&start={(page-1)*20}&as_qdr=h24&q=' + keyword)
        time.sleep(2)

        el_titles = driver.find_elements_by_css_selector("[role='heading']")  # title
        try:
            el_titles[-1].text
        except:
            print(
                "Web page exception, please check whether the web page needs to be verified, wait 5s......"
            )
            time.sleep(5)
            continue
        if el_titles[-1].text != "next page":
            break
        el_links = driver.find_elements_by_css_selector("[class='WlydOe']")  # link
        el_times = driver.find_elements_by_css_selector(
            "[class='OSrXXb ZE0LJd']"
        )  # time
        el_intros = driver.find_elements_by_css_selector("[class='iRPxbe']")  # intro
        if (
            len(el_titles) < NUM
            or len(el_links) < NUM
            or len(el_times) < NUM
            or len(el_intros) < NUM
        ):
            continue
        for link in el_links:
            # print(link)
            if link.get_attribute("href") != None:
                links.append(link.get_attribute("href"))
        # print('link nums: ', len(links))
        for e in el_times:
            times.append(e.find_element_by_tag_name("span").text)
        # print('time nums: ', len(times))
        for e in el_titles:
            if e.text not in ["", "last page", "next page"]:
                titles.append(e.text)
        # print('title nums: ', len(titles))
        for e in el_intros:
            intros.append(e.find_elements_by_tag_name("div")[2].text)
        # print(len(intros), intros)
        print(
            "link nums: ",
            len(links),
            "time nums: ",
            len(times),
            "title nums: ",
            len(titles),
            "intro nums: ",
            len(intros),
        )

        page += 1

    print(keyword + "amount of found：", len(links))
    cols = ["title", "link", "intro", "time"]
    # df = pd.DataFrame(columns=cols,data=[])
    df = pd.DataFrame(
        {
            "title": titles,
            "link": links,
            "intro": intros,
            "time": times,
        }
    )
    print(df)
    df.to_csv(f"./{keyword}_search_results/{keyword}.csv", encoding="utf_8_sig")

    print(keyword + "  - - dowload over - -")

    driver.close()

    return

In [ ]:
for keyword in keywords:
    google_search(keyword)